In [2]:
!pip install tensorflow

In [5]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop

In [4]:
def preprocess_volume(volume):
    # Clip to [-1000, 1000] HU
    volume = np.clip(volume, -1000, 1000)
    # Normalize to [0, 1]
    volume = (volume + 1000) / 2000
    return volume.astype(np.float32)


In [7]:
def augment(volume):
    # Placeholder for rotation, translation, FOV crop etc.
    # Apply random flips as example
    if np.random.rand() > 0.5:
        volume = np.flip(volume, axis=0)
    if np.random.rand() > 0.5:
        volume = np.flip(volume, axis=1)
    if np.random.rand() > 0.5:
        volume = np.flip(volume, axis=2)
    return volume


In [8]:
def load_data(num_samples=10):
    x_data = []
    y_data = []
    for _ in range(num_samples):
        # Simulated volume: shape (100, 100, 100)
        volume = np.random.uniform(-1200, 1000, size=(100, 100, 100))
        volume = preprocess_volume(volume)
        volume = augment(volume)

        # Simulated target bounding box: [x1, y1, z1, x2, y2, z2]
        bbox = np.random.uniform(0, 100, size=(6,))
        bbox = np.sort(bbox.reshape(2, 3), axis=0).flatten()  

        x_data.append(volume)
        y_data.append(bbox)

    x_data = np.expand_dims(np.array(x_data), axis=-1)  # Add channel dim: (N, 100, 100, 100, 1)
    y_data = np.array(y_data)
    return x_data, y_data


In [9]:
def build_bbnet(kernel_size=8, dense_size=16, optimizer_name='adam', learning_rate=1e-3):
    input_layer = Input(shape=(100, 100, 100, 1), name='input_volume')

    x = Conv3D(32, kernel_size, activation='relu')(input_layer)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    x = Conv3D(64, kernel_size, activation='relu')(x)
    x = MaxPooling3D(pool_size=(2, 2, 2))(x)

    x = Flatten()(x)
    x = Dense(dense_size * 2, activation='relu')(x)
    x = Dense(dense_size, activation='relu')(x)
    output_layer = Dense(6, activation='linear', name='bbox_output')(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    # Optimizer choice
    optimizers = {
        'adam': Adam(learning_rate),
        'adagrad': Adagrad(learning_rate),
        'rmsprop': RMSprop(learning_rate)
    }

    model.compile(optimizer=optimizers[optimizer_name], loss='mse', metrics=['mae'])
    return model

In [ ]:
if __name__ == "__main__":
    x_train, y_train = load_data(num_samples=100)  
    model = build_bbnet(kernel_size=8, dense_size=16, optimizer_name='adam', learning_rate=1e-3)
    
    model.summary()
    
    model.fit(x_train, y_train, batch_size=4, epochs=20, validation_split=0.2)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_volume (InputLayer)       │ (None, 100, 100, 100,  │             0 │
│                                 │ 1)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 93, 93, 93, 32) │        16,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (None, 46, 46, 46, 32) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 39, 39, 39, 64) │     1,048,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (None, 19, 19, 19, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 438976)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │    14,047,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bbox_output (Dense)             │ (None, 6)              │           102 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,112,950 (57.65 MB)

 Trainable params: 15,112,950 (57.65 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 368s 18s/step - loss: 9135.4482 - mae: 67.2912 - val_loss: 2955.6206 - val_mae: 46.9728
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 392s 19s/step - loss: 2696.3113 - mae: 43.7777 - val_loss: 1302.9707 - val_mae: 29.5049
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 367s 18s/step - loss: 1315.3253 - mae: 29.6905 - val_loss: 996.8445 - val_mae: 25.2372
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 356s 18s/step - loss: 1026.3142 - mae: 26.2399 - val_loss: 811.4250 - val_mae: 22.8125
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 353s 18s/step - loss: 877.3336 - mae: 23.6080 - val_loss: 708.0902 - val_mae: 21.0591
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 385s 19s/step - loss: 714.4538 - mae: 21.4798 - val_loss: 667.1503 - val_mae: 20.4792
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 294s 14s/step - loss: 673.4603 - mae: 20.8649 - val_loss: 610.1516 - val_mae: 19.7928
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 329s 15s/step - loss: 598.9261 - mae: 19.7274 - val_loss: 584.3537 - val_mae: 19.7101
Ep